In [5]:
"""
Example using Polars with the Iris dataset.
"""

import polars as pl
from suricata import generate_report
from IPython.display import HTML


TITANIC_URL = (
    "https://raw.githubusercontent.com/mwaskom/seaborn-data/master/titanic.csv"
)

In [6]:
# Load Iris dataset directly using Polars' CSV reader.
titanic_pdf = pl.read_csv(TITANIC_URL)
html_report = generate_report(titanic_pdf, output_file="titanic_polars_report.html")
print("Polars report saved as 'titanic_polars_report.html'")
HTML(html_report)

INFO:suricata.logger:summary_statistics executed in 0.0040 seconds
INFO:suricata.logger:missing_values executed in 0.0010 seconds
INFO:suricata.logger:correlation_matrix executed in 0.0000 seconds


Polars report saved as 'titanic_polars_report.html'
